In [1]:
# Dependencies

from config import (census_key, gkey)
import gmaps
import numpy as np
import pandas as pd

import requests
import json

import time
from scipy.stats import linregress
from matplotlib import pyplot as plt

from pprint import pprint

In [2]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)

In [3]:
city_data = pd.read_csv("../WeatherPy/Output/City_DataFrame_CSV_Output.csv",dtype="object", encoding="utf-8")
city_data.dtypes

City Name               object
Temperature (F)         object
Humidity %              object
Cloud Coverage %        object
Wind Speed (mph)        object
City Latitude           object
City Longitude          object
City Country            object
City Local Date_Time    object
Random_Latitude         object
Random_Longitude        object
dtype: object

In [4]:
# Fix data types
city_data[["City Latitude", "City Longitude",'Temperature (F)','Humidity %','Cloud Coverage %','Wind Speed (mph)']] = city_data[["City Latitude", "City Longitude",'Temperature (F)','Humidity %','Cloud Coverage %','Wind Speed (mph)']].astype(float)

city_data.dtypes

City Name                object
Temperature (F)         float64
Humidity %              float64
Cloud Coverage %        float64
Wind Speed (mph)        float64
City Latitude           float64
City Longitude          float64
City Country             object
City Local Date_Time     object
Random_Latitude          object
Random_Longitude         object
dtype: object

In [5]:
city_data

,City Name,Temperature (F),Humidity %,Cloud Coverage %,Wind Speed (mph),City Latitude,City Longitude,City Country,City Local Date_Time,Random_Latitude,Random_Longitude
0,Rikitea,77.63,71.0,96.0,11.50,-23.1203,-134.9692,pf,"Sun, 07 Feb 2021 10:13:34",-45.98,-138.65
1,Puerto Ayora,84.20,70.0,0.0,17.27,-0.7393,-90.3518,ec,"Sun, 07 Feb 2021 13:13:34",-6.42,-102.73
2,Saskylakh,-37.95,78.0,25.0,12.33,71.9167,114.0833,ru,"Mon, 08 Feb 2021 03:59:22",72.34,114.57
3,Aksarka,-16.89,88.0,100.0,29.59,66.5606,67.7975,ru,"Mon, 08 Feb 2021 00:13:34",64.74,68.78
4,Barrow,-23.80,76.0,20.0,11.50,71.2906,-156.7887,us,"Sun, 07 Feb 2021 10:11:58",69.86,-161.43
...,...,...,...,...,...,...,...,...,...,...,...
536,Honāvar,73.69,42.0,0.0,6.11,14.2833,74.4500,in,"Mon, 08 Feb 2021 00:45:12",13.61,73.01
537,Camaná,75.99,70.0,77.0,8.05,-16.6228,-72.7111,pe,"Sun, 07 Feb 2021 14:15:12",-20.86,-76.24
538,Kolondiéba,87.85,15.0,100.0,5.79,11.0882,-6.8926,ml,"Sun, 07 Feb 2021 19:15:12",10.93,-6.91
539,Buluang,75.49,87.0,100.0,5.99,13.3068,123.3425,ph,"Mon, 08 Feb 2021 03:15:13",13.43,116.32


In [6]:
locations = city_data[["City Latitude", "City Longitude"]]
humidity  = city_data["Humidity %"]

In [7]:
humidity.describe()

count    541.000000
mean      71.020333
std       20.823721
min        9.000000
25%       61.000000
50%       76.000000
75%       86.000000
max      100.000000
Name: Humidity %, dtype: float64

In [8]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False,opacity=.7, 
                                 max_intensity=100,point_radius = 2.5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [43]:
# restrict the number of cities, originally 72 - 80
ideal_cities = city_data[((city_data["Temperature (F)"]) >= 72) & ((city_data["Temperature (F)"]) <= 80) & ((city_data["Wind Speed (mph)"]) >= 5) & ((city_data["Wind Speed (mph)"]) <= 10)& ((city_data["Cloud Coverage %"]) >= 0) & ((city_data["Cloud Coverage %"]) <= 10)]

In [51]:
ideal_cities.count()

City Name               16
Temperature (F)         16
Humidity %              16
Cloud Coverage %        16
Wind Speed (mph)        16
City Latitude           16
City Longitude          16
City Country            16
City Local Date_Time    16
Random_Latitude         16
Random_Longitude        16
dtype: int64

In [52]:
ideal_cities.head(3)

,City Name,Temperature (F),Humidity %,Cloud Coverage %,Wind Speed (mph),City Latitude,City Longitude,City Country,City Local Date_Time,Random_Latitude,Random_Longitude
96,Isangel,77.54,79.0,9.0,5.75,-19.5500,169.2667,vu,"Mon, 08 Feb 2021 06:05:37",-19.49,175.8
102,Ati,72.97,18.0,0.0,8.41,13.2154,18.3353,td,"Sun, 07 Feb 2021 20:13:51",13.78,18.44
115,Pisco,73.40,83.0,0.0,9.22,-13.7000,-76.2167,pe,"Sun, 07 Feb 2021 14:09:23",-22.26,-87.15


In [68]:
# Part 1 set up to look up geocoordinates and add hotel info
ideal_cities['Hotel_Name'] = ""
ideal_cities['Hotel_Address'] = ""
ideal_cities['Hotel_Lat'] = ""
ideal_cities['Hotel_Lng'] = ""
ideal_cities.head()

,City Name,Temperature (F),Humidity %,Cloud Coverage %,Wind Speed (mph),City Latitude,City Longitude,City Country,City Local Date_Time,Random_Latitude,Random_Longitude,Hotel_Name,Hotel_Address,Hotel_Lat,Hotel_Lng
96,Isangel,77.54,79.0,9.0,5.75,-19.5500,169.2667,vu,"Mon, 08 Feb 2021 06:05:37",-19.49,175.8,,,,
102,Ati,72.97,18.0,0.0,8.41,13.2154,18.3353,td,"Sun, 07 Feb 2021 20:13:51",13.78,18.44,,,,
115,Pisco,73.40,83.0,0.0,9.22,-13.7000,-76.2167,pe,"Sun, 07 Feb 2021 14:09:23",-22.26,-87.15,,,,
124,Matara,75.47,87.0,7.0,5.19,5.9485,80.5353,lk,"Mon, 08 Feb 2021 00:43:54",0.73,80.7,,,,
126,Chui,75.25,62.0,1.0,9.93,-33.6971,-53.4616,uy,"Sun, 07 Feb 2021 16:13:54",-60.76,-26.96,,,,


In [86]:
# Part 2
for index, row in ideal_cities.iterrows():
    # set up a parameters dictionary
    lat = row["City Latitude"]
    lng = row["City Longitude"]

    params = {
        "location": f"{lat},{lng}",
        "radius": "5000",
        "types": "lodging",
        "key": gkey
    }
    

# base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # print(f"{lat},{lng}")
# run a request using our params dictionary
    response = requests.get(base_url, params=params)
    places_data = response.json()
    try:
        ideal_cities.loc[index,'Hotel_Name'] = places_data["results"][0]["name"]
        ideal_cities.loc[index,'Hotel_Address'] = places_data["results"][0]["vicinity"]
        ideal_cities.loc[index,'Hotel_Lat'] = places_data['results'][0]['geometry']['location']['lat']
        ideal_cities.loc[index,'Hotel_Lng'] = places_data['results'][0]['geometry']['location']['lng']
            
        # print(row["Hotel_Name"])
        # print(row["Hotel_Address"])
        # print(row["City Name"])
        # print(row["City Country"])
        # print(row["Hotel_Lat"])
        # print(row["Hotel_Lng"])
        # print(f"\n\n")

        # print(places_data["results"][0]["name"])
        # print(row["City Name"])
        # print(row["City Country"])
        # print(places_data["results"][0]["vicinity"])
        # print(places_data['results'][0]['geometry']['location']['lat'])
        # print(places_data['results'][0]['geometry']['location']['lng'])
        # print(f"\n\n")

    except (KeyError, IndexError):
        print(f"For City: {ideal_cities.loc[index,'City Name']}, Missing field/result... deleting.")
        ideal_cities.drop(index, inplace=True)

For City: Ati, Missing field/result... deleting.



For City: Beloha, Missing field/result... deleting.





In [ ]:
# Create a marker_layer using the poverty list to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
    info_box_content=[f"Poverty Rate: {rate}" for rate in poverty_rate])
fig.add_layer(markers)
fig